# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/cities_luz.csv"
cities_df = pd.read_csv(file_path, index_col=0)
cities_df

,City Id,City Name,Country,Latitude,Longitude,Datetime,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed
0,964432,Port Alfred,ZA,-33.59,26.89,1603691397,68.40,79,97,16.58
1,2377457,Nouadhibou,MR,20.93,-17.03,1603691397,71.60,68,0,19.46
2,2155415,New Norfolk,AU,-42.78,147.06,1603691397,57.99,49,0,1.99
3,5122534,Jamestown,US,42.10,-79.24,1603691230,44.01,93,90,9.17
4,1106677,Bambous Virieux,MU,-20.34,57.76,1603691176,78.80,65,40,18.34
...,...,...,...,...,...,...,...,...,...,...
554,72181,Marzuq,YE,14.40,46.47,1603691528,74.70,17,20,2.91
555,2121025,Srednekolymsk,RU,67.45,153.68,1603691528,12.22,88,0,1.48
556,2236967,Soyo,AO,-6.13,12.37,1603691529,77.00,94,75,7.70
557,927643,Machinga,MW,-15.17,35.30,1603691529,78.37,51,69,7.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
#From Heatmap documentation
#This piece of code was taken from gmaps documentation
locations = cities_df[['Latitude', 'Longitude']]
weights = cities_df['Humidity (%)']
fig = gmaps.figure(center=(0, 0), zoom_level=1)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#My ideal temperature to take vacations is from 80 to 87 Fahrenheit degrees
#Cloudiness from 5 to 10% (I don't like too sunny places!)
#Wind speed < 10 mph
ideal_vac_cities_df = cities_df.loc[(cities_df['Max Temp (F)']>77) & (cities_df['Max Temp (F)']<90) 
                                & (cities_df['Cloudiness (%)']>0) & (cities_df['Cloudiness (%)']<15)
                                & (cities_df['Wind Speed']<10)
                                   ]
ideal_vac_cities_df



,City Id,City Name,Country,Latitude,Longitude,Datetime,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed
38,1282256,Hithadhoo,MV,-0.60,73.08,1603691407,83.79,67,6,8.19
136,1266503,Kiri Buru,IN,22.08,85.35,1603691432,84.22,36,2,6.93
170,5847411,Kahului,US,20.89,-156.47,1603691440,82.40,65,1,9.17
203,3985710,Cabo San Lucas,MX,22.89,-109.91,1603691365,79.00,54,5,8.05
258,3353383,Rundu,NaN,-17.93,19.77,1603691462,80.24,22,14,8.08
260,4564070,Culebra,PR,18.30,-65.30,1603691462,82.40,69,1,8.05
293,5850554,Makakilo City,US,21.35,-158.09,1603691275,80.60,74,1,1.68
450,1262230,Nāgar Karnūl,IN,16.48,78.32,1603691504,83.77,52,3,7.58
497,2274275,Robertsport,LR,6.75,-11.37,1603691515,78.78,82,1,5.32
548,1626542,Sorong,ID,-0.88,131.25,1603691527,83.97,73,5,1.54


In [6]:
#Dropping rows in case there are null values
ideal_vac_cities_df.dropna()

,City Id,City Name,Country,Latitude,Longitude,Datetime,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed
38,1282256,Hithadhoo,MV,-0.60,73.08,1603691407,83.79,67,6,8.19
136,1266503,Kiri Buru,IN,22.08,85.35,1603691432,84.22,36,2,6.93
170,5847411,Kahului,US,20.89,-156.47,1603691440,82.40,65,1,9.17
203,3985710,Cabo San Lucas,MX,22.89,-109.91,1603691365,79.00,54,5,8.05
260,4564070,Culebra,PR,18.30,-65.30,1603691462,82.40,69,1,8.05
293,5850554,Makakilo City,US,21.35,-158.09,1603691275,80.60,74,1,1.68
450,1262230,Nāgar Karnūl,IN,16.48,78.32,1603691504,83.77,52,3,7.58
497,2274275,Robertsport,LR,6.75,-11.37,1603691515,78.78,82,1,5.32
548,1626542,Sorong,ID,-0.88,131.25,1603691527,83.97,73,5,1.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Copying the clean ideal cities for vacation in the holel DataFrame hotel_df
hotel_df = ideal_vac_cities_df.copy()
#Adding "Hotel Name" column with empty values
hotel_df['Hotel Name']=''

In [8]:
#Setting general parameters to search for hotels with 5000 meters radius in each city
#For hotels, Google's target type is 'lodging'
target_type = "lodging"
radius = 5000

#Using Google Places nearby option as we're searching for hotels in a certain area
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [9]:
hotel_df

,City Id,City Name,Country,Latitude,Longitude,Datetime,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Hotel Name
38,1282256,Hithadhoo,MV,-0.60,73.08,1603691407,83.79,67,6,8.19,
136,1266503,Kiri Buru,IN,22.08,85.35,1603691432,84.22,36,2,6.93,
170,5847411,Kahului,US,20.89,-156.47,1603691440,82.40,65,1,9.17,
203,3985710,Cabo San Lucas,MX,22.89,-109.91,1603691365,79.00,54,5,8.05,
258,3353383,Rundu,NaN,-17.93,19.77,1603691462,80.24,22,14,8.08,
260,4564070,Culebra,PR,18.30,-65.30,1603691462,82.40,69,1,8.05,
293,5850554,Makakilo City,US,21.35,-158.09,1603691275,80.60,74,1,1.68,
450,1262230,Nāgar Karnūl,IN,16.48,78.32,1603691504,83.77,52,3,7.58,
497,2274275,Robertsport,LR,6.75,-11.37,1603691515,78.78,82,1,5.32,
548,1626542,Sorong,ID,-0.88,131.25,1603691527,83.97,73,5,1.54,


In [10]:
#Iterating in each row to get hotel with 5000 radius of city's latitude and longitude
for row in hotel_df.itertuples(index=False):
    #print(row[3])
    coords = f"{row[3]}, {row[4]}"
    params = {
        "location": coords,
        "types" : target_type,
        "radius": radius,
        "key": g_key
    }
    #Managing exceptions as there may be places without query results
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[hotel_df['City Id'] == row[0], 'Hotel Name'] = response["results"][0]["name"]
        #hotel_df['Hotel'] = response["results"][0]["name"]
        #print(response["results"][0]["name"])
    except:  
        hotel_df.loc[hotel_df['City Id'] == row[0], 'Hotel Name'] = 'No Hotel Found'
        print(f"MISSING VALUES: There are no search results for hotels in {row[1]}")
        continue

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
#marker_locations = locations
#fig.add_layer(gmaps.heatmap_layer(locations, weights=hotel_df['Humidity (%)']))
#fig

# Display Map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))